# Training DenseNet121 Over CIFAR10 dataset

In [ ]:
import keras
import os
from skimage import io
import numpy as np
from keras import backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, DepthwiseConv2D
from keras.layers import Concatenate
from keras.models import load_model
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger
from keras.callbacks import Callback
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as k
import copy
import time

In [ ]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 85
l = 6 # no of layers in dense block
num_filter = 35 # growth rate k
compression = 1.0
dropout_rate = 0.20

# CCP Augmented Training
ccp_augment = False #For augmented CCP attack training dataset, change ccp_augment=False to ccp_augment=True

# Preprocessing the Dataset

In [ ]:
# CCP ATTACK FUNCTIONS
def change_brightness(image, alpha, beta):
  new_image = np.zeros(image.shape, np.int64)
  new_image = np.clip( alpha*image + beta, 0, 255)
  return new_image
	
def CCP_Attack_Brightness(image, transform):
	img = copy.copy(image)
	for channel in range(img.shape[2]):
		temp1 = image[:,:,0]
		temp2 = image[:,:,1]
		temp3 = image[:,:,2]

		temp = temp1 * transform[channel][0] + temp2 * transform[channel][1] + temp3 * transform[channel][2]

		img[:,:,channel] = temp/3

	img1 = change_brightness(img, 2, 0)
	return img1

In [ ]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# CCP Augmentation
# CCP Augmentation
if ccp_augment:
  print('CCP augmented included!')
  # Copy Dataset
  x_train_trans = copy.copy(x_train)

  # Intializing at each transformation [0,1.0]
  a = np.random.uniform(low=0.0, high=1.0, size=(3,))
  b = np.random.uniform(low=0.0, high=1.0, size=(3,))
  c = np.random.uniform(low=0.0, high=1.0, size=(3,))
  transform = np.array([a,b,c]) 


  for test in range(len(x_train)):
    
    ## Comment or Uncomment this for Fixed or Variable settings respectively.
    a = np.random.uniform(low=0.0, high=1.0, size=(3,))
    b = np.random.uniform(low=0.0, high=1.0, size=(3,))
    c = np.random.uniform(low=0.0, high=1.0, size=(3,))
    transform = np.array([a,b,c])

    ## Copy Image
    img = copy.copy(x_train[test])

    ## CCP Attack
    x_train_trans[test] = CCP_Attack_Brightness(img, transform)

  # Including augmented training dataset, i.e., 1,00,000 training images total
  x_train = np.vstack((x_train,x_train_trans))
  y_train = np.vstack((y_train,y_train))
    

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

def preprocess_data(data_set):
    mean = np.array([125.3, 123.0, 113.9])
    std = np.array([63.0, 62.1, 66.7])

    data_set -= mean
    data_set /= std
    return data_set

x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10)

In [ ]:
# Data augementation
datagen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True,
    fill_mode='nearest',
    zoom_range=0.10
)

datagen_train.fit(x_train)

#Model Building

In [ ]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [ ]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [ ]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [ ]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

# Callbacks and Model Compiling

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 5, min_lr = 0.000001)

early_stop = EarlyStopping(monitor = "val_loss", patience = 10)

def decay_fn(epoch, lr):
    if epoch < 50:
        return 0.001
    elif epoch >= 50 and epoch < 75:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(decay_fn)

csv_logger = CSVLogger('training.log')

filepath = "{epoch:03d}-{val_acc:.3f}.hdf5"
model_chkpt = ModelCheckpoint(filepath, monitor = "val_loss", save_best_only=True, verbose = 1)

In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),#SGD(lr=0.1, momentum=0.9, decay=0.0001, nesterov=True),
              metrics=['accuracy'])

# Training the Model

In [ ]:
history = model.fit_generator(
    datagen_train.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=(len(x_train)/batch_size)*5,
    epochs=epochs,
    verbose = 1,
    validation_data=(x_val, y_val),
    callbacks = [lr_scheduler, csv_logger, model_chkpt]
)

model.save('densenet_cifar10.h5')

# Evaluate Model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Testing on CCP Attack
### Testing the CIFAR10 test dataset with s=2 and b=0
#### (CCP_F and CCP_V)

In [ ]:
# Load Model
import keras
new_model = keras.models.load_model('densenet_cifar10.h5')

In [ ]:
def preprocess_data(data_set):
    mean = np.array([125.3, 123.0, 113.9])
    std = np.array([63.0, 62.1, 66.7])

    data_set -= mean
    data_set /= std
    return data_set

In [ ]:
# Test Over CCP

# Load Dataset
(x_train1, y_train1), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

# Copy Test Set
x_test1_trans = copy.copy(x_test1)
# nois = np.random.randint(low=0, high=256, size=(32, 32, 3))
a = np.random.uniform(low=0.0, high=1.0, size=(3,))
b = np.random.uniform(low=0.0, high=1.0, size=(3,))
c = np.random.uniform(low=0.0, high=1.0, size=(3,))
transform1 = np.array([a,b,c]) 

for test in range(len(x_test1_trans)):

  ## Comment or Uncomment below for Fixed or Variable settings respectively.
  # a = np.random.uniform(low=0.0, high=1.0, size=(3,))
  # b = np.random.uniform(low=0.0, high=1.0, size=(3,))
  # c = np.random.uniform(low=0.0, high=1.0, size=(3,))
  # transform1 = np.array([a,b,c])

  # Copy the test image
  img = copy.copy(x_test1_trans[test])

  # CCP Attack
  x_test1_trans[test] = CCP_Attack_Brightness(img,transform1)


# Convert to FLOAT32
x_test1_trans = x_test1_trans.astype('float32')

# Preprocess Transformed Test Data for testing
x_test1_trans = preprocess_data(x_test1_trans)

# Convert class vectors to binary class matrices.
y_test1 = keras.utils.to_categorical(y_test1, 10)


# Evaluate the model
scores = new_model.evaluate(x_test1_trans, y_test1, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])